In [1]:
import glob
import pandas as pd
import os
import glob
from datetime import datetime,timedelta

# Personnal Import 
from utilities import get_mode_date2path,DataSet
from load_data import load_subway_15_min,load_CRITER,load_netmob

## Dataset : Metro 15 minutes 2019 2020
- On va travailler sur les mois de Mars, Avril, Mai 2019, avec les 4 lignes de Metro A, B, C, D ainsi que la ligne de Funiculaire '326'.
- On intègrera les données NetMob dans la suite.

Init:

In [2]:
time_step_per_hour=4
H,W,D = 6,1,1
step_ahead = 1
start,end = datetime(2019,3,16),datetime(2019,6,1)

reindex = pd.date_range(start,end,freq = f'{60/time_step_per_hour}min')
print(f'Number of time-slot: {4*24*(end-start).days}')

Number of time-slot: 7392


In [3]:
try : 
    df_metro = pd.read_csv('data/Metro_15min_mar2019_mai2019.csv',index_col = 0)
    df_metro.datetime = pd.to_datetime(df_metro.datetime)

except:
    folder_path = '../../Data/keolis_data_2019-2020/'
    txt_path = "Métro 15 minutes 2019 2020.txt"

    df_metro_funi_2019_2020 = load_subway_15_min(folder_path+txt_path)
    df_metro = df_metro_funi_2019_2020[(df_metro_funi_2019_2020.datetime >= start)&(df_metro_funi_2019_2020.datetime < end)]

df_metro.head()

,datetime,Station,Code ligne,in,out
288362,2019-03-16 00:00:00,Ampère Victor Hugo,A,34,30.0
288363,2019-03-16 00:15:00,Ampère Victor Hugo,A,40,34.0
288364,2019-03-16 00:30:00,Ampère Victor Hugo,A,17,14.0
288365,2019-03-16 00:45:00,Ampère Victor Hugo,A,1,16.0
288366,2019-03-16 01:00:00,Ampère Victor Hugo,A,0,0.0


# Build Feature Vector : 
### We can: 
- select a single station, reindex and fillna. Keep 'in' and 'out' as two features
- Thanks to 'pivot_lane', we can re-organize the dataframe and keep only 'in' or only 'out' (or both).


#### Select only one station :

In [4]:
ampere = df_metro[df_metro.Station == df_metro.Station.iloc[0]]
ampere = ampere.set_index('datetime').sort_index().reindex(reindex).fillna(0)[['in','out']]
ampere.head()

,in,out
2019-03-16 00:00:00,34.0,30.0
2019-03-16 00:15:00,40.0,34.0
2019-03-16 00:30:00,17.0,14.0
2019-03-16 00:45:00,1.0,16.0
2019-03-16 01:00:00,0.0,0.0


In [5]:
dataset = DataSet(ampere,time_step_per_hour=time_step_per_hour)
norm_dataset = dataset.normalize()
(U,Utarget,df_verif) =  norm_dataset.get_feature_vect(step_ahead,H,D,W)
df_verif.head()

,t-672,t-96,t-6,t-5,t-4,t-3,t-2,t-1,t
672,2019-03-16 00:00:00,2019-03-22 00:00:00,2019-03-22 22:30:00,2019-03-22 22:45:00,2019-03-22 23:00:00,2019-03-22 23:15:00,2019-03-22 23:30:00,2019-03-22 23:45:00,2019-03-23 00:00:00
673,2019-03-16 00:15:00,2019-03-22 00:15:00,2019-03-22 22:45:00,2019-03-22 23:00:00,2019-03-22 23:15:00,2019-03-22 23:30:00,2019-03-22 23:45:00,2019-03-23 00:00:00,2019-03-23 00:15:00
674,2019-03-16 00:30:00,2019-03-22 00:30:00,2019-03-22 23:00:00,2019-03-22 23:15:00,2019-03-22 23:30:00,2019-03-22 23:45:00,2019-03-23 00:00:00,2019-03-23 00:15:00,2019-03-23 00:30:00
675,2019-03-16 00:45:00,2019-03-22 00:45:00,2019-03-22 23:15:00,2019-03-22 23:30:00,2019-03-22 23:45:00,2019-03-23 00:00:00,2019-03-23 00:15:00,2019-03-23 00:30:00,2019-03-23 00:45:00
676,2019-03-16 01:00:00,2019-03-22 01:00:00,2019-03-22 23:30:00,2019-03-22 23:45:00,2019-03-23 00:00:00,2019-03-23 00:15:00,2019-03-23 00:30:00,2019-03-23 00:45:00,2019-03-23 01:00:00


#### Select all the station :
Charpennes peut concerner métro B comme métro A. Comme on est pas en capacité de savoir quelle ligne l'usagé à choisi, on somme les deux lignes sur la même station.

In [6]:
subway_in = pd.pivot_table(df_metro,index = 'datetime',columns = 'Station',values = 'in',aggfunc = 'sum', fill_value = 0).reindex(reindex).fillna(0)
subway_out = pd.pivot_table(df_metro,index = 'datetime',columns = 'Station',values = 'out',aggfunc = 'sum', fill_value = 0).reindex(reindex).fillna(0)
subway_in.head()

Station,Ampère Victor Hugo,Bellecour,Brotteaux,Charpennes,Cordeliers,Croix Paquet,Croix-Rousse,Cuire,Cusset,Debourg,...,Part-Dieu,Perrache,Place Guichard,Place Jean Jaurès,République Villeurbanne,Sans Souci,Saxe - Gambetta,Stade de Gerland,Valmy,Vieux Lyon
2019-03-16 00:00:00,34.0,396.0,37.0,164.0,143.0,6.0,33.0,17.0,24.0,40.0,...,68.0,144.0,26.0,48.0,53.0,55.0,112.0,59.0,71.0,186.0
2019-03-16 00:15:00,40.0,298.0,56.0,164.0,139.0,3.0,46.0,7.0,20.0,46.0,...,77.0,145.0,31.0,32.0,45.0,55.0,159.0,56.0,41.0,179.0
2019-03-16 00:30:00,17.0,258.0,24.0,74.0,100.0,19.0,6.0,0.0,4.0,13.0,...,43.0,55.0,15.0,19.0,4.0,45.0,98.0,4.0,72.0,153.0
2019-03-16 00:45:00,1.0,44.0,3.0,13.0,13.0,1.0,6.0,3.0,2.0,2.0,...,6.0,5.0,3.0,4.0,0.0,13.0,17.0,4.0,3.0,14.0
2019-03-16 01:00:00,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,4.0,4.0,2.0,2.0,0.0,0.0,1.0,0.0


In [9]:
dataset = DataSet(subway_in,time_step_per_hour=time_step_per_hour)
norm_dataset = dataset.normalize()
colname2indx,indx2colname = norm_dataset.bijection_name_indx()
(U,Utarget,df_verif) =  norm_dataset.get_feature_vect(step_ahead,H,D,W)

U.shape, Utarget.shape

(torch.Size([6721, 41, 8]), torch.Size([6721, 41, 1]))

### colname2indx allow to keep track on the position of a station ('Ampere', 'Brotteaux', ...) within the Tensor

## Build Adjacency Matrix : 

## Load Models and Train 
- Essaye de capturer des dépendances spatio-temporelle. 
- Créer un modèle avec un module d'attention ou Transformer

NameError: name 'random' is not defined

## Ajout des données NetMob

## Concentration sur les événements atypiques / anomalies